In [1]:
import pandas as pd

In [2]:
# en_10k.txt와 ko_5.8k.xlsx를 읽어와서 각각 en_df, ko_df에 저장
en_df = pd.read_csv('en_10k.txt', header=None)
ko_df = pd.read_excel('ko_5.8k.xlsx')

display(en_df.head())
display(ko_df.head())

,0
0,the
1,of
2,and
3,to
4,a


,순위,단어,품사,풀이,등급
0,1.0,것01,의,NaN,A
1,2.0,하다01,동,NaN,A
2,3.0,있다01,보,NaN,A
3,4.0,있다01,형,NaN,A
4,5.0,되다01,동,어른이 ~,A


In [3]:
used_ko_part = ['고', '대', '동', '명', '부', '형']
print(len(ko_df))
ko_df = ko_df[ko_df['품사'].isin(used_ko_part)]
print(len(ko_df))

5965
5628


In [4]:
# ko_df의 '단어' 칼럼의 텍스트 내 숫자가 들어가는 경우 숫자를 제거
ko_df['단어'] = ko_df['단어'].str.replace('\d+', '', regex=True)
display(ko_df.head())

,순위,단어,품사,풀이,등급
1,2.0,하다,동,NaN,A
3,4.0,있다,형,NaN,A
4,5.0,되다,동,어른이 ~,A
7,8.0,나,대,NaN,A
9,10.0,없다,형,NaN,A


In [7]:
ko_df = ko_df[['단어']]
ko_df.drop_duplicates(inplace=True)
ko_df = ko_df.reset_index(drop=True)
display(ko_df.head())
print(len(ko_df))

,단어
0,하다
1,있다
2,되다
3,나
4,없다


5351


In [8]:
ko_df.columns = ['ko']
en_df.columns = ['en']
display(ko_df.head())
display(en_df.head())

,ko
0,하다
1,있다
2,되다
3,나
4,없다


,en
0,the
1,of
2,and
3,to
4,a


In [9]:
ko_df.to_csv('ko_5.8k.csv', index=False)
en_df.to_csv('en_10k.csv', index=False)